# 金価格予測 - 日次実行ノートブック

**モデル:** メタモデル Attempt 7 (XGBoost + 8サブモデル + ポジションサイジング)

**テストセット性能:**
- 方向的中率 (DA): 60.04%
- 高確信度DA (HCDA): 64.13%
- シャープレシオ: 2.46 (取引コスト5bps控除後)

**使い方:**
1. 下のセルで `TARGET_DATE` を設定（空欄なら本日基準）
2. 「Run All Cells」で全セル実行
3. 最下部の予測レポートを確認

In [ ]:
# ================================================================
# ユーザー入力: 予測したい日付を設定
# ================================================================
# 書式: 'YYYY-MM-DD'（例: '2026-02-18'）
# None または '' のままにすると、本日を基準に翌営業日を予測
TARGET_DATE = None

# ================================================================
# 設定パラメータ
# ================================================================
FRED_API_KEY = None           # FRED APIキー（Noneなら.envから自動読込）
TOTAL_CAPITAL = 1_000_000     # 運用総資本（円 or ドル）
MAX_POSITION_PCT = 0.30       # 最大ポジション比率（資本の30%）
MIN_POSITION_PCT = 0.05       # 最小ポジション比率（資本の5%）
COST_BPS = 5                  # 取引コスト（ベーシスポイント）

In [ ]:
# ================================================================
# 1. ライブラリ読込 & 初期設定
# ================================================================
import pandas as pd
import numpy as np
import xgboost as xgb
import warnings
import os
from datetime import datetime, timedelta

warnings.filterwarnings('ignore')
np.random.seed(42)

# --- FRED API（米経済指標取得用）---
try:
    from fredapi import Fred
except ImportError:
    import subprocess
    subprocess.run(['pip', 'install', 'fredapi'], check=True)
    from fredapi import Fred

# --- HMM（隠れマルコフモデル：サブモデルで使用）---
try:
    from hmmlearn.hmm import GaussianHMM
except ImportError:
    import subprocess
    subprocess.run(['pip', 'install', 'hmmlearn'], check=True)
    from hmmlearn.hmm import GaussianHMM

# --- GMM（ガウス混合モデル：VIXサブモデルで使用）---
from sklearn.mixture import GaussianMixture

# --- Yahoo Finance（価格データ取得）---
import yfinance as yf

# FRED APIキーの読込
if FRED_API_KEY is None:
    try:
        from dotenv import load_dotenv
        load_dotenv()
        FRED_API_KEY = os.getenv('FRED_API_KEY')
    except ImportError:
        pass

if FRED_API_KEY is None:
    raise ValueError('FRED_API_KEY が未設定です。このセルで直接指定するか、.envファイルに設定してください。')

fred = Fred(api_key=FRED_API_KEY)

# 予測日の解決
if TARGET_DATE is None or TARGET_DATE == '':
    target_date = pd.Timestamp.now().normalize()
else:
    target_date = pd.Timestamp(TARGET_DATE)

# データ取得期間（予測日の数日先まで取得して最新データを確保）
fetch_end = (target_date + timedelta(days=5)).strftime('%Y-%m-%d')
fetch_start = '2014-01-01'  # モデル学習開始日

print(f'予測基準日: {target_date.strftime("%Y-%m-%d")}（翌営業日の金リターンを予測）')
print(f'データ取得範囲: {fetch_start} ～ {fetch_end}')
print(f'XGBoost: {xgb.__version__}')

## 2. 生データ取得
13のAPIソースから市場データを取得します:
- **Yahoo Finance**: 金先物(GC=F), ドル指数(DXY), GLD ETF, 銀(SI=F), 銅(HG=F), SKEW, GVZ, TNX, IRX, FVX
- **FRED API**: 実質金利(DFII10), VIX(VIXCLS), 10年国債(DGS10), 2年国債(DGS2), BEI(T10YIE)

In [ ]:
# ================================================================
# 2. 生データ取得（全13ソース）
# ================================================================
print('=' * 60)
print('生データ取得中...')
print('=' * 60)

# --- 金先物（COMEX GC=F）---
print('金先物 (GC=F) を取得中...')
gold_raw = yf.download('GC=F', start=fetch_start, end=fetch_end, progress=False)
gold_close = gold_raw['Close'].squeeze()
gold_close.index = pd.to_datetime(gold_close.index).tz_localize(None)
gold_return = gold_close.pct_change() * 100          # 日次リターン（%）
gold_return_next = gold_return.shift(-1)              # 翌日リターン（= 予測ターゲット）
print(f'  金: {len(gold_close)}行, 最新: {gold_close.index[-1].strftime("%Y-%m-%d")}')

# --- ドル指数（DXY）--- 金と逆相関の強い指標
print('ドル指数 (DX-Y.NYB) を取得中...')
dxy_raw = yf.download('DX-Y.NYB', start=fetch_start, end=fetch_end, progress=False)
dxy_close = dxy_raw['Close'].squeeze()
dxy_close.index = pd.to_datetime(dxy_close.index).tz_localize(None)
print(f'  DXY: {len(dxy_close)}行')

# --- GLD ETF --- テクニカル・ETFフロー分析に使用
print('GLD ETF を取得中...')
gld_raw = yf.Ticker('GLD').history(start=fetch_start, end=fetch_end, auto_adjust=True)
gld_raw.index = gld_raw.index.tz_localize(None)
print(f'  GLD: {len(gld_raw)}行')

# --- 銀・銅先物 --- クロスアセット分析に使用
print('銀先物 (SI=F) / 銅先物 (HG=F) を取得中...')
silver_raw = yf.download('SI=F', start=fetch_start, end=fetch_end, progress=False)
copper_raw = yf.download('HG=F', start=fetch_start, end=fetch_end, progress=False)
silver_close = silver_raw['Close'].squeeze()
copper_close = copper_raw['Close'].squeeze()
silver_close.index = pd.to_datetime(silver_close.index).tz_localize(None)
copper_close.index = pd.to_datetime(copper_close.index).tz_localize(None)
print(f'  銀: {len(silver_close)}行, 銅: {len(copper_close)}行')

# --- FRED: 実質金利（10年TIPS利回り）--- 金と最も強い負の相関
print('実質金利 DFII10 を取得中...')
real_rate_raw = fred.get_series('DFII10', observation_start=fetch_start)
real_rate_raw.index = pd.to_datetime(real_rate_raw.index)
print(f'  DFII10: {len(real_rate_raw)}行')

# --- FRED: VIX --- リスクオフ指標
print('VIX (VIXCLS) を取得中...')
vix_raw = fred.get_series('VIXCLS', observation_start=fetch_start)
vix_raw.index = pd.to_datetime(vix_raw.index)
print(f'  VIX: {len(vix_raw)}行')

# --- FRED: イールドカーブ（10年-2年スプレッド）---
print('国債利回り DGS10, DGS2 を取得中...')
dgs10_raw = fred.get_series('DGS10', observation_start=fetch_start)
dgs2_raw = fred.get_series('DGS2', observation_start=fetch_start)
dgs10_raw.index = pd.to_datetime(dgs10_raw.index)
dgs2_raw.index = pd.to_datetime(dgs2_raw.index)
yield_spread = dgs10_raw - dgs2_raw   # イールドスプレッド
print(f'  DGS10: {len(dgs10_raw)}行, DGS2: {len(dgs2_raw)}行')

# --- FRED: ブレークイーブンインフレ率（BEI）---
print('BEI (T10YIE) を取得中...')
ie_raw = fred.get_series('T10YIE', observation_start=fetch_start)
ie_raw.index = pd.to_datetime(ie_raw.index)
print(f'  T10YIE: {len(ie_raw)}行')

# --- Yahoo: オプション市場指標 --- SKEW（テールリスク）とGVZ（金ボラティリティ）
print('SKEW, GVZ を取得中...')
skew_data = yf.Ticker('^SKEW').history(start=fetch_start, end=fetch_end, auto_adjust=True)
gvz_data = yf.Ticker('^GVZ').history(start=fetch_start, end=fetch_end, auto_adjust=True)
skew_data.index = skew_data.index.tz_localize(None)
gvz_data.index = gvz_data.index.tz_localize(None)
print(f'  SKEW: {len(skew_data)}行, GVZ: {len(gvz_data)}行')

# --- Yahoo: 債券利回り代理指標 --- イールドカーブサブモデル用
print('TNX, IRX, FVX を取得中...')
tnx = yf.Ticker('^TNX').history(start=fetch_start, end=fetch_end)   # 10年国債利回り
irx = yf.Ticker('^IRX').history(start=fetch_start, end=fetch_end)   # 13週Tビル利回り
fvx = yf.Ticker('^FVX').history(start=fetch_start, end=fetch_end)   # 5年国債利回り
tnx.index = tnx.index.tz_localize(None)
irx.index = irx.index.tz_localize(None)
fvx.index = fvx.index.tz_localize(None)
print(f'  TNX: {len(tnx)}行, IRX: {len(irx)}行, FVX: {len(fvx)}行')

print('\n全データ取得完了。')

## 3. サブモデル特徴量の計算
8つのサブモデルが市場のレジーム・パターンを抽出:

| # | サブモデル | 手法 | 出力特徴量 |
|---|-----------|------|-----------|
| 1 | VIX | GMM(2成分) | レジーム確率, 平均回帰z, 持続性 |
| 2 | テクニカル | HMM(2状態) | トレンドレジーム, 平均回帰z, ボラレジーム |
| 3 | クロスアセット | HMM(3状態) | レジーム確率, 景気後退シグナル, 乖離度 |
| 4 | イールドカーブ | HMM(2状態) | スプレッド速度z, 曲率z |
| 5 | ETFフロー | HMM(4状態) | レジーム確率, 資本集中度, PV乖離 |
| 6 | インフレ期待 | HMM(3状態) | レジーム確率, アンカリングz, 金感応度z |
| 7 | オプション | HMM(2状態) | リスクレジーム確率 |
| 8 | 時間コンテキスト | Transformer | コンテキストスコア |

In [ ]:
# ================================================================
# ヘルパー関数: HMMの複数リスタートフィッティング
# ================================================================
def fit_hmm_best(X, n_components, covariance_type='full', n_restarts=10, n_iter=200, seeds=None):
    """複数の乱数シードでHMMを学習し、対数尤度最大のモデルを返す。
    HMMは初期値に敏感なため、複数回リスタートが必要。"""
    if seeds is None:
        seeds = list(range(n_restarts))
    best_model = None
    best_score = -np.inf
    for seed in seeds:
        try:
            model = GaussianHMM(
                n_components=n_components,
                covariance_type=covariance_type,
                n_iter=n_iter,
                random_state=seed,
            )
            model.fit(X)
            score = model.score(X)
            if score > best_score:
                best_score = score
                best_model = model
        except Exception:
            continue
    return best_model


def hmm_highest_var_state(model):
    """HMMの状態の中で、共分散行列のトレース（総分散）が最大の状態を返す。
    = 最もボラタイルなレジーム（高変動状態）のインデックス。"""
    n_comp = model.n_components
    traces = []
    for i in range(n_comp):
        if model.covariance_type == 'full':
            traces.append(np.trace(model.covars_[i]))
        elif model.covariance_type == 'diag':
            traces.append(np.sum(model.covars_[i]))
        else:
            traces.append(np.sum(model.covars_[i]))
    return np.argmax(traces)


print('ヘルパー関数定義完了。')

In [ ]:
# ================================================================
# 3a. VIXサブモデル
# ================================================================
# VIXの対数変化率にGMM(2成分)をフィット → 高変動レジームの確率を抽出
# 出力: vix_regime_probability（高変動状態の確率）
#        vix_mean_reversion_z（VIX水準の平均回帰z値）
#        vix_persistence（VIX変化の自己相関 = 持続性）
print('VIXサブモデルを計算中...')

vix_series = vix_raw.dropna()
vix_log_change = np.log(vix_series).diff().dropna()  # 対数変化率

# GMM（2成分ガウス混合モデル）で高変動/低変動レジームを分離
gmm_vix = GaussianMixture(n_components=2, covariance_type='diag', n_init=3, max_iter=100, random_state=42)
gmm_vix.fit(vix_log_change.values.reshape(-1, 1))

# 高分散の成分（= 高変動レジーム）のインデックスを特定
high_var_idx = np.argmax(gmm_vix.covariances_.flatten())
probs = gmm_vix.predict_proba(vix_log_change.values.reshape(-1, 1))
vix_regime_prob = pd.Series(probs[:, high_var_idx], index=vix_log_change.index, name='vix_regime_probability')

# 平均回帰zスコア（40日移動平均・標準偏差で標準化）
vix_z = (vix_series - vix_series.rolling(40).mean()) / vix_series.rolling(40).std()
vix_z = vix_z.clip(-4, 4)
vix_z.name = 'vix_mean_reversion_z'

# 持続性 = 30日ローリング自己相関（ラグ1）
vix_persistence = vix_log_change.rolling(30).apply(
    lambda x: pd.Series(x).autocorr(lag=1), raw=False
)
vix_persistence.name = 'vix_persistence'

vix_submodel = pd.DataFrame({
    'vix_regime_probability': vix_regime_prob,
    'vix_mean_reversion_z': vix_z,
    'vix_persistence': vix_persistence,
})
print(f'  VIXサブモデル: {len(vix_submodel)}行, NaN: {vix_submodel.isna().sum().to_dict()}')

In [ ]:
# ================================================================
# 3b. テクニカルサブモデル
# ================================================================
# GLD ETFの[リターン, GK-Volatility]にHMM(2状態)をフィット
# 出力: tech_trend_regime_prob（高変動レジーム確率）
#        tech_mean_reversion_z（リターンの平均回帰z値）
#        tech_volatility_regime（GK-Volのz値 = ボラティリティレジーム）
print('テクニカルサブモデルを計算中...')

gld_close = gld_raw['Close']
gld_open = gld_raw['Open']
gld_high = gld_raw['High']
gld_low = gld_raw['Low']

gld_returns = gld_close.pct_change()  # GLD日次リターン

# Garman-Klass ボラティリティ（日中高安を使った効率的なボラ推定）
gk_vol = np.sqrt(
    0.5 * np.log(gld_high / gld_low) ** 2
    - (2 * np.log(2) - 1) * np.log(gld_close / gld_open) ** 2
).clip(lower=1e-8)

# HMM（2状態, フル共分散）: [リターン, GK-Vol] → トレンド/ボラレジーム分離
tech_input = pd.DataFrame({'returns': gld_returns, 'gk_vol': gk_vol}).dropna()
X_tech = tech_input.values

hmm_tech = fit_hmm_best(X_tech, n_components=2, covariance_type='full', n_restarts=10)
high_var_state = hmm_highest_var_state(hmm_tech)
tech_probs = hmm_tech.predict_proba(X_tech)
tech_regime_prob = pd.Series(tech_probs[:, high_var_state], index=tech_input.index, name='tech_trend_regime_prob')

# 平均回帰zスコア（15日ウィンドウ）
tech_z = (gld_returns - gld_returns.rolling(15).mean()) / gld_returns.rolling(15).std()
tech_z = tech_z.clip(-4, 4)
tech_z.name = 'tech_mean_reversion_z'

# ボラティリティレジーム zスコア（60日ウィンドウ）
gk_vol_z = (gk_vol - gk_vol.rolling(60).mean()) / gk_vol.rolling(60).std()
gk_vol_z = gk_vol_z.clip(-4, 4)
gk_vol_z.name = 'tech_volatility_regime'

tech_submodel = pd.DataFrame({
    'tech_trend_regime_prob': tech_regime_prob,
    'tech_mean_reversion_z': tech_z,
    'tech_volatility_regime': gk_vol_z,
})
print(f'  テクニカルサブモデル: {len(tech_submodel)}行')

In [ ]:
# ================================================================
# 3c. クロスアセットサブモデル
# ================================================================
# 金・銀・銅のリターンにHMM(3状態)をフィット → 市場レジーム分離
# 出力: xasset_regime_prob（高変動レジーム確率）
#        xasset_recession_signal（金/銅比率のz変化 = 景気後退シグナル）
#        xasset_divergence（金-銀リターン差のz値 = 相対乖離度）
print('クロスアセットサブモデルを計算中...')

gold_ret = gold_close.pct_change()
silver_ret = silver_close.pct_change()
copper_ret = copper_close.pct_change()

# 共通日付で揃える
xasset_df = pd.DataFrame({
    'gold_ret': gold_ret,
    'silver_ret': silver_ret,
    'copper_ret': copper_ret,
    'gold_close': gold_close,
    'copper_close': copper_close,
    'silver_close': silver_close,
}).dropna()

# HMM（3状態, フル共分散）: 全データでフィット
X_xasset = xasset_df[['gold_ret', 'silver_ret', 'copper_ret']].values
hmm_xasset = fit_hmm_best(X_xasset, n_components=3, covariance_type='full', n_restarts=10)
high_var_state_xa = hmm_highest_var_state(hmm_xasset)
xa_probs = hmm_xasset.predict_proba(X_xasset)
xasset_regime_prob = pd.Series(xa_probs[:, high_var_state_xa], index=xasset_df.index, name='xasset_regime_prob')

# 景気後退シグナル: 金/銅比率のz値の変化（金高・銅安 = リスクオフ）
gc_ratio = xasset_df['gold_close'] / xasset_df['copper_close']
gc_z = (gc_ratio - gc_ratio.rolling(90).mean()) / gc_ratio.rolling(90).std()
xasset_recession = gc_z.diff().clip(-4, 4)
xasset_recession.name = 'xasset_recession_signal'

# 乖離度: 金-銀リターンスプレッドのz値（20日ウィンドウ）
gs_diff = xasset_df['gold_ret'] - xasset_df['silver_ret']
gs_z = (gs_diff - gs_diff.rolling(20).mean()) / gs_diff.rolling(20).std()
xasset_divergence = gs_z.clip(-4, 4)
xasset_divergence.name = 'xasset_divergence'

xasset_submodel = pd.DataFrame({
    'xasset_regime_prob': xasset_regime_prob,
    'xasset_recession_signal': xasset_recession,
    'xasset_divergence': xasset_divergence,
})
print(f'  クロスアセットサブモデル: {len(xasset_submodel)}行')

In [ ]:
# ================================================================
# 3d. イールドカーブサブモデル
# ================================================================
# Yahoo TNX/IRX/FVXを利回り代理指標として使用
# 出力: yc_spread_velocity_z（スプレッド変化速度のz値）
#        yc_curvature_z（カーブ曲率変化のz値）
print('イールドカーブサブモデルを計算中...')

# Yahoo Financeの利回りデータを実際の利回り(%)に変換
dgs10_yf = tnx['Close'] / 100.0                      # 10年国債利回り
dgs2_yf = irx['Close'] / 100.0 * (365.0 / 91.0)     # 13週Tビル → 年率換算
dgs5_yf = fvx['Close'] / 100.0                        # 5年国債利回り

yc_df = pd.DataFrame({'dgs10': dgs10_yf, 'dgs2': dgs2_yf, 'dgs5': dgs5_yf}).dropna()
yc_spread = yc_df['dgs10'] - yc_df['dgs2']            # 10年-2年スプレッド
yc_curvature = yc_df['dgs5'] - 0.5 * (yc_df['dgs2'] + yc_df['dgs10'])  # バタフライ曲率

dgs10_change = yc_df['dgs10'].diff()
dgs2_change = yc_df['dgs2'].diff()

# HMM（2状態, 対角共分散）: 利回り変化でレジーム分離
yc_hmm_input = pd.DataFrame({'dgs10_chg': dgs10_change, 'dgs2_chg': dgs2_change}).dropna()
X_yc = yc_hmm_input.values

# 先頭70%で学習（未来リーク防止）
n_train_yc = int(len(X_yc) * 0.70)
hmm_yc = fit_hmm_best(X_yc[:n_train_yc], n_components=2, covariance_type='diag',
                       n_restarts=5, seeds=[0, 42, 123, 456, 789])
high_var_state_yc = hmm_highest_var_state(hmm_yc)
yc_probs = hmm_yc.predict_proba(X_yc)
yc_regime_prob = pd.Series(yc_probs[:, high_var_state_yc], index=yc_hmm_input.index, name='yc_regime_prob')

# スプレッド速度z値（5日変化の30日z標準化）
spread_change_5 = yc_spread.diff(5)
yc_spread_vel_z = (spread_change_5 - spread_change_5.rolling(30).mean()) / spread_change_5.rolling(30).std()
yc_spread_vel_z = yc_spread_vel_z.clip(-4, 4)
yc_spread_vel_z.name = 'yc_spread_velocity_z'

# 曲率z値（1日変化の120日z標準化）
curvature_change = yc_curvature.diff()
yc_curv_z = (curvature_change - curvature_change.rolling(120).mean()) / curvature_change.rolling(120).std()
yc_curv_z = yc_curv_z.clip(-4, 4)
yc_curv_z.name = 'yc_curvature_z'

yc_submodel = pd.DataFrame({
    'yc_spread_velocity_z': yc_spread_vel_z,
    'yc_curvature_z': yc_curv_z,
})
print(f'  イールドカーブサブモデル: {len(yc_submodel)}行')

In [ ]:
# ================================================================
# 3e. ETFフローサブモデル
# ================================================================
# GLD ETFの出来高パターンからHMM(4状態)で資金流入レジームを推定
# 出力: etf_regime_prob（蓄積状態の確率 = 機関投資家の買い増し局面）
#        etf_capital_intensity（ドル建て出来高のz値 = 資本流入強度）
#        etf_pv_divergence（価格-出来高相関のz値 = PV乖離）
print('ETFフローサブモデルを計算中...')

gld_vol = gld_raw['Volume']
gld_cl = gld_raw['Close']

dollar_volume = gld_cl * gld_vol                       # ドル建て出来高
volume_ma20 = gld_vol.rolling(20).mean()               # 20日移動平均出来高
log_volume_ratio = np.log(gld_vol / volume_ma20).replace([np.inf, -np.inf], 0)
gld_ret = gld_cl.pct_change()

# HMM（4状態, フル共分散）: [出来高比, 金リターン] → 蓄積/分配レジーム分離
etf_input = pd.DataFrame({'lvr': log_volume_ratio, 'gold_ret': gold_ret}).reindex(gld_raw.index)
etf_input['gold_ret'] = gold_ret.reindex(gld_raw.index)
etf_input = etf_input.dropna()
X_etf = etf_input.values

# 先頭70%で学習
n_train_etf = int(len(X_etf) * 0.70)
hmm_etf = fit_hmm_best(X_etf[:n_train_etf], n_components=4, covariance_type='full',
                        n_restarts=11, seeds=list(range(42, 53)))

# 蓄積状態 = 学習期間中で金リターン平均が最も高い状態
train_states = hmm_etf.predict(X_etf[:n_train_etf])
train_gold_ret_by_state = pd.Series(X_etf[:n_train_etf, 1]).groupby(train_states).mean()
accum_state = train_gold_ret_by_state.idxmax()

etf_probs = hmm_etf.predict_proba(X_etf)
etf_regime_prob = pd.Series(etf_probs[:, accum_state], index=etf_input.index, name='etf_regime_prob')

# 資本集中度 z値（60日ウィンドウ）
cap_z = (dollar_volume - dollar_volume.rolling(60).mean()) / dollar_volume.rolling(60).std()
cap_z = cap_z.replace([np.inf, -np.inf], 0)
cap_z.name = 'etf_capital_intensity'

# PV乖離: 価格変化と出来高変化の相関のz値
vol_changes = gld_vol.pct_change()
rolling_corr = gld_ret.rolling(10).corr(vol_changes)
pv_div = (rolling_corr - rolling_corr.rolling(40).mean()) / rolling_corr.rolling(40).std()
pv_div = pv_div.replace([np.inf, -np.inf], 0)
pv_div.name = 'etf_pv_divergence'

etf_submodel = pd.DataFrame({
    'etf_regime_prob': etf_regime_prob,
    'etf_capital_intensity': cap_z,
    'etf_pv_divergence': pv_div,
})
print(f'  ETFフローサブモデル: {len(etf_submodel)}行')

In [ ]:
# ================================================================
# 3f. インフレ期待サブモデル
# ================================================================
# ブレークイーブンインフレ率(T10YIE)の変化とボラにHMM(3状態)をフィット
# 出力: ie_regime_prob（高変動レジーム確率）
#        ie_anchoring_z（インフレ期待のアンカリング度合い = ボラの異常度）
#        ie_gold_sensitivity_z（インフレ期待と金リターンの短期相関z値）
print('インフレ期待サブモデルを計算中...')

ie_change = ie_raw.diff().dropna()          # BEIの日次変化
ie_vol_5d = ie_change.rolling(5).std()      # 5日ボラティリティ

# HMM（3状態, フル共分散）: [BEI変化, 5日ボラ]
ie_hmm_input = pd.DataFrame({'ie_chg': ie_change, 'ie_vol': ie_vol_5d}).dropna()
X_ie = ie_hmm_input.values

# 先頭70%で学習
n_train_ie = int(len(X_ie) * 0.70)
hmm_ie = fit_hmm_best(X_ie[:n_train_ie], n_components=3, covariance_type='full',
                       n_restarts=3, seeds=[42, 43, 44])
high_var_state_ie = hmm_highest_var_state(hmm_ie)
ie_probs = hmm_ie.predict_proba(X_ie)
ie_regime_prob = pd.Series(ie_probs[:, high_var_state_ie], index=ie_hmm_input.index, name='ie_regime_prob')

# アンカリングz値 = 5日ボラの120日z標準化（高い = インフレ期待が不安定）
ie_anchor_z = (ie_vol_5d - ie_vol_5d.rolling(120).mean()) / ie_vol_5d.rolling(120).std()
ie_anchor_z = ie_anchor_z.clip(-4, 4).replace([np.inf, -np.inf], 0)
ie_anchor_z.name = 'ie_anchoring_z'

# 金感応度z値 = BEI変化と金リターンの短期相関（5日）のz標準化
gold_ret_for_ie = gold_ret.reindex(ie_change.index)
ie_gold_corr = ie_change.rolling(5).corr(gold_ret_for_ie)
ie_sens_z = (ie_gold_corr - ie_gold_corr.rolling(40).mean()) / ie_gold_corr.rolling(40).std()
ie_sens_z = ie_sens_z.clip(-4, 4).replace([np.inf, -np.inf], 0)
ie_sens_z.name = 'ie_gold_sensitivity_z'

ie_submodel = pd.DataFrame({
    'ie_regime_prob': ie_regime_prob,
    'ie_anchoring_z': ie_anchor_z,
    'ie_gold_sensitivity_z': ie_sens_z,
})
print(f'  インフレ期待サブモデル: {len(ie_submodel)}行')

In [ ]:
# ================================================================
# 3g. オプション市場サブモデル
# ================================================================
# SKEWインデックス（テールリスク指標）とGVZ（金ボラティリティ）の
# 変化率にHMM(2状態)をフィット → リスクレジーム確率を抽出
# 出力: options_risk_regime_prob（高リスクレジーム確率）
print('オプション市場サブモデルを計算中...')

skew_change = skew_data['Close'].diff()    # SKEW日次変化
gvz_change = gvz_data['Close'].diff()      # GVZ日次変化

opt_input = pd.DataFrame({'skew_chg': skew_change, 'gvz_chg': gvz_change}).dropna()
X_opt = opt_input.values

# 先頭70%で学習
n_train_opt = int(len(X_opt) * 0.70)
hmm_opt = GaussianHMM(
    n_components=2, covariance_type='full', n_iter=100, random_state=42
)
hmm_opt.fit(X_opt[:n_train_opt])

high_var_state_opt = hmm_highest_var_state(hmm_opt)
opt_probs = hmm_opt.predict_proba(X_opt)
options_regime_prob = pd.Series(opt_probs[:, high_var_state_opt], index=opt_input.index,
                                 name='options_risk_regime_prob')

opt_submodel = pd.DataFrame({'options_risk_regime_prob': options_regime_prob})
print(f'  オプション市場サブモデル: {len(opt_submodel)}行')

In [ ]:
# ================================================================
# 3h. 時間コンテキストサブモデル（PyTorch Transformer）
# ================================================================
# 本来はTransformerモデルの重み(model.pt)で推論するが、
# ローカルに重みファイルがないため、事前計算済みCSVまたは
# デフォルト値 0.5（= 最大不確実性）を使用。
# NaN補完値として0.5を使っているため、メタモデルは0.5入力に対して
# 適切にキャリブレーション済み。

# 事前計算CSVを複数のパスで探索
tc_csv_path = os.path.join(
    os.path.dirname(os.path.abspath('.')),
    'data', 'dataset_upload_clean', 'temporal_context.csv'
)

tc_paths = [
    tc_csv_path,
    '../data/dataset_upload_clean/temporal_context.csv',
    'data/dataset_upload_clean/temporal_context.csv',
]

tc_loaded = False
for p in tc_paths:
    if os.path.exists(p):
        tc_df = pd.read_csv(p)
        tc_df['date'] = pd.to_datetime(tc_df['date'])
        tc_df = tc_df.set_index('date')
        tc_loaded = True
        print(f'  時間コンテキスト: CSVから読込 ({p}, {len(tc_df)}行)')
        break

if not tc_loaded:
    print('  時間コンテキスト: CSVなし → デフォルト値0.5（中立）を使用')
    tc_df = pd.DataFrame(index=gold_close.index)
    tc_df['temporal_context_score'] = 0.5

tc_submodel = tc_df[['temporal_context_score']]

## 4. 特徴量マトリクスの組立
5つのベース特徴量 + 8サブモデルからの19特徴量 = 合計24次元の特徴量ベクトルを構築。
NaN補完ルール: レジーム確率→0.5, z値→0.0, 連続値→中央値

In [ ]:
# ================================================================
# 4. 特徴量マトリクスの組立（メタモデル Attempt 7 と同一構成）
# ================================================================
print('=' * 60)
print('特徴量マトリクスを組立中...')
print('=' * 60)

# --- ベース特徴量の構築 ---
base_df = pd.DataFrame(index=gold_close.index)
base_df['gold_return_next'] = gold_return_next  # 予測ターゲット（翌日リターン%）

# 原系列を取り込み（差分計算用）
base_df['_real_rate'] = real_rate_raw.reindex(base_df.index)
base_df['_dxy'] = dxy_close.reindex(base_df.index)
base_df['_vix'] = vix_raw.reindex(base_df.index)
base_df['_yield_spread'] = yield_spread.reindex(base_df.index)
base_df['_ie'] = ie_raw.reindex(base_df.index)

# 前方補完（FRED系は営業日のみ更新のためギャップあり）
base_df = base_df.ffill()

# 定常化のための差分変換
base_df['real_rate_change'] = base_df['_real_rate'].diff()       # 実質金利変化
base_df['dxy_change'] = base_df['_dxy'].diff()                   # ドル指数変化
base_df['vix'] = base_df['_vix']                                  # VIX水準（元々定常的）
base_df['yield_spread_change'] = base_df['_yield_spread'].diff()  # スプレッド変化
base_df['inflation_exp_change'] = base_df['_ie'].diff()           # BEI変化

# ヘルパー列を削除
base_df = base_df.drop(columns=['_real_rate', '_dxy', '_vix', '_yield_spread', '_ie'])

# --- サブモデル特徴量の結合 ---
for name, sub_df in [
    ('vix', vix_submodel),
    ('technical', tech_submodel),
    ('cross_asset', xasset_submodel),
    ('yield_curve', yc_submodel),
    ('etf_flow', etf_submodel),
    ('inflation_exp', ie_submodel),
    ('options', opt_submodel),
    ('temporal', tc_submodel),
]:
    base_df = base_df.join(sub_df, how='left')

print(f'結合後: {base_df.shape}')

# --- 特徴量カラム定義（Attempt 7 と同一順序）---
FEATURE_COLUMNS = [
    'real_rate_change', 'dxy_change', 'vix', 'yield_spread_change', 'inflation_exp_change',
    'vix_regime_probability', 'vix_mean_reversion_z', 'vix_persistence',
    'tech_trend_regime_prob', 'tech_mean_reversion_z', 'tech_volatility_regime',
    'xasset_regime_prob', 'xasset_recession_signal', 'xasset_divergence',
    'yc_spread_velocity_z', 'yc_curvature_z',
    'etf_regime_prob', 'etf_capital_intensity', 'etf_pv_divergence',
    'ie_regime_prob', 'ie_anchoring_z', 'ie_gold_sensitivity_z',
    'options_risk_regime_prob',
    'temporal_context_score',
]
TARGET = 'gold_return_next'

# --- NaN補完（学習時と同一ルール）---
# レジーム確率 → 0.5（中立 = 「レジーム不明」）
regime_cols = ['vix_regime_probability', 'tech_trend_regime_prob', 'xasset_regime_prob',
               'etf_regime_prob', 'ie_regime_prob', 'options_risk_regime_prob', 'temporal_context_score']
for col in regime_cols:
    if col in base_df.columns:
        base_df[col] = base_df[col].fillna(0.5)

# z値 → 0.0（平均 = 「異常なし」）
z_cols = ['vix_mean_reversion_z', 'tech_mean_reversion_z', 'yc_spread_velocity_z',
          'yc_curvature_z', 'etf_capital_intensity', 'etf_pv_divergence',
          'ie_anchoring_z', 'ie_gold_sensitivity_z']
for col in z_cols:
    if col in base_df.columns:
        base_df[col] = base_df[col].fillna(0.0)

# シグナル/乖離 → 0.0（中立）
div_cols = ['xasset_recession_signal', 'xasset_divergence']
for col in div_cols:
    if col in base_df.columns:
        base_df[col] = base_df[col].fillna(0.0)

# 連続値 → 中央値
cont_cols = ['tech_volatility_regime', 'vix_persistence']
for col in cont_cols:
    if col in base_df.columns:
        base_df[col] = base_df[col].fillna(base_df[col].median())

# ベース特徴量にNaNがある行は除外（最初の数日分）
base_df = base_df.dropna(subset=['real_rate_change', 'dxy_change', 'vix',
                                   'yield_spread_change', 'inflation_exp_change'])

# 全特徴量の存在確認
missing = [c for c in FEATURE_COLUMNS if c not in base_df.columns]
if missing:
    raise ValueError(f'特徴量が不足: {missing}')

print(f'最終データセット: {len(base_df)}行, {len(FEATURE_COLUMNS)}特徴量')
print(f'日付範囲: {base_df.index.min().strftime("%Y-%m-%d")} ～ {base_df.index.max().strftime("%Y-%m-%d")}')
print(f'残存NaN: {base_df[FEATURE_COLUMNS].isna().sum().sum()}')

## 5. XGBoostメタモデル学習（Attempt 7 アーキテクチャ）
- 時系列分割: Train 70% / Val 15% / Test 15%（シャッフルなし）
- ハイパーパラメータ: Attempt 2 フォールバック基準値（Optuna 100試行で検証済み）
- Bootstrapアンサンブル: 5モデル（信頼度推定用）
- OLSスケーリング: バリデーションセットで較正

In [ ]:
# ================================================================
# 5. Train/Val/Test分割 + XGBoost学習
# ================================================================
print('=' * 60)
print('メタモデル学習中（Attempt 7 アーキテクチャ）...')
print('=' * 60)

# ターゲットが存在する行のみ（最終行は翌日リターンが未定義）
trainable = base_df.dropna(subset=[TARGET]).copy()

# 時系列分割: 70/15/15（シャッフルなし = 未来リーク防止）
n_total = len(trainable)
n_train = int(n_total * 0.70)
n_val = int(n_total * 0.15)

train_data = trainable.iloc[:n_train]
val_data = trainable.iloc[n_train:n_train + n_val]
test_data = trainable.iloc[n_train + n_val:]

X_train = train_data[FEATURE_COLUMNS].values
y_train = train_data[TARGET].values
X_val = val_data[FEATURE_COLUMNS].values
y_val = val_data[TARGET].values
X_test = test_data[FEATURE_COLUMNS].values
y_test = test_data[TARGET].values

print(f'Train: {len(train_data)}行 ({train_data.index[0].strftime("%Y-%m-%d")} ～ {train_data.index[-1].strftime("%Y-%m-%d")})')
print(f'Val:   {len(val_data)}行 ({val_data.index[0].strftime("%Y-%m-%d")} ～ {val_data.index[-1].strftime("%Y-%m-%d")})')
print(f'Test:  {len(test_data)}行 ({test_data.index[0].strftime("%Y-%m-%d")} ～ {test_data.index[-1].strftime("%Y-%m-%d")})')

# --- Attempt 7 ハイパーパラメータ（Attempt 2 フォールバック値）---
# Optuna 100試行の最良トライアル(#89)とほぼ同等の性能
MODEL_PARAMS = {
    'objective': 'reg:squarederror',  # 二乗誤差回帰
    'max_depth': 2,                    # 浅い木 → 過学習抑制
    'min_child_weight': 14,            # 最小葉サンプル数
    'reg_lambda': 4.76,                # L2正則化
    'reg_alpha': 3.65,                 # L1正則化
    'subsample': 0.478,                # 行サブサンプリング
    'colsample_bytree': 0.371,         # 列サブサンプリング
    'learning_rate': 0.025,            # 学習率
    'tree_method': 'hist',             # ヒストグラムベース
    'eval_metric': 'rmse',
    'verbosity': 0,
    'seed': 42,
}

# プライマリモデル学習（早期停止付き）
print('\nプライマリモデル学習中...')
model = xgb.XGBRegressor(**MODEL_PARAMS, n_estimators=300, early_stopping_rounds=100)
model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)

# Bootstrapアンサンブル（5モデル: 乱数シード違い → 信頼度推定用）
print('Bootstrapアンサンブル学習中（5モデル）...')
bootstrap_models = []
for seed in [42, 43, 44, 45, 46]:
    params_boot = MODEL_PARAMS.copy()
    params_boot['seed'] = seed
    m = xgb.XGBRegressor(**params_boot, n_estimators=300, early_stopping_rounds=100)
    m.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)
    bootstrap_models.append(m)

# OLSスケーリング係数（バリデーションセットで較正）
# 予測値を実際の変動幅にスケールするための線形回帰係数
pred_val = model.predict(X_val)
alpha_ols = np.clip(np.sum(pred_val * y_val) / (np.sum(pred_val ** 2) + 1e-10), 0.5, 10.0)
print(f'OLSスケーリング係数: {alpha_ols:.3f}')

# --- テストセットでのサニティチェック ---
pred_test = model.predict(X_test)
mask = (y_test != 0) & (pred_test != 0)
test_da = (np.sign(pred_test[mask]) == np.sign(y_test[mask])).mean()
test_mae = np.mean(np.abs(pred_test * alpha_ols - y_test))

# シャープレシオ（取引コスト控除後）
positions = np.sign(pred_test)
strat_ret = positions * y_test / 100.0
pos_chg = np.abs(np.diff(positions, prepend=0))
net_ret = strat_ret - pos_chg * (COST_BPS / 10000.0)
test_sharpe = (net_ret.mean() / net_ret.std()) * np.sqrt(252) if net_ret.std() > 0 else 0

print(f'\nテストセット性能:')
print(f'  方向的中率(DA): {test_da * 100:.2f}%')
print(f'  MAE(スケール後): {test_mae:.4f}%')
print(f'  シャープレシオ:  {test_sharpe:.2f}')
print(f'\nモデル学習完了。')

## 6. 予測日の翌営業日リターンを予測

In [ ]:
# ================================================================
# 6. 予測日の翌営業日リターンを予測
# ================================================================
print('=' * 60)
print(f'{target_date.strftime("%Y-%m-%d")} の予測を生成中...')
print('=' * 60)

# 予測日（またはその直前の営業日）をデータセットから特定
available_dates = base_df.index[base_df.index <= target_date]
if len(available_dates) == 0:
    raise ValueError(f'{target_date.strftime("%Y-%m-%d")} 以前のデータがありません。')

actual_date = available_dates[-1]
if actual_date != target_date:
    print(f'注意: {target_date.strftime("%Y-%m-%d")} は非営業日です。')
    print(f'      直前の営業日 {actual_date.strftime("%Y-%m-%d")} を使用します。')

# 特徴量ベクトルを抽出
X_pred = base_df.loc[[actual_date], FEATURE_COLUMNS].values

# プライマリ予測
pred_raw = model.predict(X_pred)[0]             # 生の予測値
pred_scaled = pred_raw * alpha_ols               # OLSスケール後

# Bootstrap予測（5モデルの予測の分散 → 信頼度推定）
boot_preds = np.array([m.predict(X_pred)[0] for m in bootstrap_models])
boot_mean = boot_preds.mean()
boot_std = boot_preds.std()                      # 小さいほど高信頼
boot_confidence = 1.0 / (1.0 + boot_std)

# 方向
direction = 'UP' if pred_raw > 0 else 'DOWN'
direction_jp = '上昇' if pred_raw > 0 else '下落'

# 確信度レベル（テストセットでの予測値分布に基づくパーセンタイル）
abs_pred = abs(pred_raw)
if abs_pred > np.percentile(np.abs(model.predict(X_test)), 80):
    confidence_level = 'HIGH'
    confidence_level_jp = '高'
elif abs_pred > np.percentile(np.abs(model.predict(X_test)), 50):
    confidence_level = 'MEDIUM'
    confidence_level_jp = '中'
else:
    confidence_level = 'LOW'
    confidence_level_jp = '低'

# 金価格コンテキスト
latest_gold_price = gold_close.loc[:actual_date].iloc[-1]
expected_price_change = latest_gold_price * pred_scaled / 100

print(f'\n{"=" * 50}')
print(f'  基準日:           {actual_date.strftime("%Y-%m-%d")}')
print(f'  予測対象:         翌営業日の金リターン')
print(f'  予測方向:         {direction} ({direction_jp})')
print(f'  生リターン:       {pred_raw:+.4f}%')
print(f'  スケール後:       {pred_scaled:+.4f}%')
print(f'  確信度:           {confidence_level} ({confidence_level_jp})')
print(f'  Bootstrap Std:    {boot_std:.4f}')
print(f'  Bootstrap 信頼度: {boot_confidence:.4f}')
print(f'  ---')
print(f'  現在の金価格:     ${latest_gold_price:.2f}')
print(f'  予測変動額:       ${expected_price_change:+.2f}')
print(f'  予測価格:         ${latest_gold_price + expected_price_change:.2f}')
print(f'{"=" * 50}')

## 7. ポジションサイジング
予測の確信度に応じて、資本の5%～30%でポジションサイズを動的に調整。
確信度スコア = 0.6 × 予測パーセンタイル + 0.4 × Bootstrap信頼度パーセンタイル

In [ ]:
# ================================================================
# 7. ポジションサイジング（確信度加重）
# ================================================================
print('=' * 60)
print('ポジションサイジング')
print('=' * 60)

# --- ポジションサイジング戦略 ---
# 2つの信頼度指標の加重平均でポジション比率を決定:
#   1. |予測値|の大きさ（テストセット内でのパーセンタイル順位）
#   2. Bootstrap一致度（5モデルの予測のばらつきの小ささ）

# テストセットでのキャリブレーション
test_pred_abs = np.abs(model.predict(X_test))
test_boot_preds = np.array([m.predict(X_test) for m in bootstrap_models])
test_boot_std = np.std(test_boot_preds, axis=0)
test_boot_conf = 1.0 / (1.0 + test_boot_std)

# 今回の予測のパーセンタイル順位
pred_pctile = (test_pred_abs < abs_pred).mean()        # 予測値の強さ
conf_pctile = (test_boot_conf < boot_confidence).mean() # Bootstrap信頼度

# 統合確信度スコア（0～1）
combined_confidence = 0.6 * pred_pctile + 0.4 * conf_pctile

# ポジション比率にマッピング（MIN～MAX）
position_pct = MIN_POSITION_PCT + (MAX_POSITION_PCT - MIN_POSITION_PCT) * combined_confidence
position_value = TOTAL_CAPITAL * position_pct

# COMEX金先物の契約数換算
gold_oz_per_contract = 100  # COMEX GC: 100オンス/契約
gold_contract_value = latest_gold_price * gold_oz_per_contract
n_contracts = position_value / gold_contract_value if gold_contract_value > 0 else 0

# 取引コスト見積もり
trade_cost = position_value * (COST_BPS / 10000.0)

# 期待損益
expected_pnl = position_value * pred_scaled / 100
expected_pnl_after_cost = expected_pnl - trade_cost

print(f'\n  運用資本:         ${TOTAL_CAPITAL:,.0f}')
print(f'  統合確信度:       {combined_confidence:.1%}')
print(f'    予測パーセンタイル: {pred_pctile:.1%}')
print(f'    Boot パーセンタイル: {conf_pctile:.1%}')
print(f'')
print(f'  ポジション比率:   {position_pct:.1%}（{position_pct * 100:.1f}%）')
print(f'  ポジション額:     ${position_value:,.0f}')
print(f'  方向:             {"ロング（買い）" if pred_raw > 0 else "ショート（売り）"}')
print(f'  COMEX契約数:      {n_contracts:.2f}枚（概算）')
print(f'')
print(f'  予測リターン:     {pred_scaled:+.4f}%')
print(f'  期待損益:         ${expected_pnl:+,.0f}')
print(f'  取引コスト:       ${trade_cost:,.0f}')
print(f'  純期待損益:       ${expected_pnl_after_cost:+,.0f}')

# リスク警告
worst_case_loss = position_value * 0.94 / 100  # MAEベース（平均予測誤差）
tail_risk_loss = position_value * 3.0 / 100    # テールリスク（~3%の極端な変動）
print(f'\n  リスク見積もり:')
print(f'    平均予測誤差(MAE): ${worst_case_loss:,.0f}（金0.94%変動時）')
print(f'    テールリスク(3%):  ${tail_risk_loss:,.0f}（極端な変動時）')

## 8. 特徴量診断
予測日の各特徴量の値とモデルにおける重要度を表示。重要度が高い特徴量ほど予測に大きな影響を与えています。

In [ ]:
# ================================================================
# 8. 特徴量の値と重要度（診断用）
# ================================================================
print(f'{actual_date.strftime("%Y-%m-%d")} の特徴量値:\n')

feat_vals = base_df.loc[actual_date, FEATURE_COLUMNS]

# モデルの特徴量重要度（gain）
importances = model.feature_importances_
feat_df = pd.DataFrame({
    'Feature': FEATURE_COLUMNS,
    'Value': feat_vals.values,
    'Importance': importances,
}).sort_values('Importance', ascending=False)

# 重要度順に表示（バーチャートつき）
for _, row in feat_df.iterrows():
    bar = '#' * int(row['Importance'] * 100)
    print(f'  {row["Feature"]:30s} = {row["Value"]:+10.4f}  重要度={row["Importance"]:.3f} {bar}')

print(f'\n特徴量数: {len(FEATURE_COLUMNS)}')

## 9. 予測サマリー
最終レポート。過去の日付を指定した場合は、実際の翌日リターンとの照合結果も表示します。

In [ ]:
# ================================================================
# 9. 予測サマリー + 過去日付の場合は実績照合
# ================================================================
from datetime import timedelta

# 翌営業日の推定（土日スキップ）
next_day = actual_date + timedelta(days=1)
if next_day.weekday() == 5:    # 土曜 → 月曜
    next_day += timedelta(days=2)
elif next_day.weekday() == 6:  # 日曜 → 月曜
    next_day += timedelta(days=1)

pos_direction = 'ロング（買い）' if pred_raw > 0 else 'ショート（売り）'

# --- 実績照合（過去日付の場合）---
# actual_date の翌日リターンがデータに存在するか確認
actual_return = None
actual_direction_jp = None
is_correct = None
actual_next_price = None

if actual_date in base_df.index:
    val = base_df.loc[actual_date, 'gold_return_next']
    if not pd.isna(val):
        actual_return = val
        actual_direction_jp = '上昇' if actual_return > 0 else '下落'
        # 方向の正否判定（予測と実績の符号が一致していれば正解）
        if pred_raw != 0 and actual_return != 0:
            is_correct = (pred_raw > 0) == (actual_return > 0)
        # 翌日の実際の金価格
        next_dates = gold_close.index[gold_close.index > actual_date]
        if len(next_dates) > 0:
            actual_next_price = gold_close.loc[next_dates[0]]

# --- レポート出力 ---
print()
print('=' * 62)
print(f'  金価格予測レポート')
print('=' * 62)
print(f'  基準日:         {actual_date.strftime("%Y-%m-%d")}（{actual_date.strftime("%A")}）')
print(f'  予測対象日:     {next_day.strftime("%Y-%m-%d")}（{next_day.strftime("%A")}）')
print('-' * 62)
print(f'  予測方向:       {direction_jp}')
print(f'  予測リターン:   {pred_scaled:+.4f}%')
print(f'  確信度:         {confidence_level_jp}（{confidence_level}）')
print(f'  Bootstrap Std:  {boot_std:.4f}')
print('-' * 62)
print(f'  現在の金価格:   ${latest_gold_price:,.2f}')
print(f'  予測変動額:     ${expected_price_change:+,.2f}')
print(f'  予測価格:       ${latest_gold_price + expected_price_change:,.2f}')

# --- 実績照合セクション（過去日付のみ表示）---
if actual_return is not None:
    actual_change_dollar = latest_gold_price * actual_return / 100
    print('-' * 62)
    print(f'  【実績照合】')
    print(f'  実際の方向:     {actual_direction_jp}')
    print(f'  実際のリターン: {actual_return:+.4f}%')
    print(f'  実際の変動額:   ${actual_change_dollar:+,.2f}')
    if actual_next_price is not None:
        print(f'  実際の翌日価格: ${actual_next_price:,.2f}')
    print(f'  予測誤差(MAE):  {abs(pred_scaled - actual_return):.4f}%')
    if is_correct is not None:
        result_mark = '○ 正解' if is_correct else '× 不正解'
        print(f'  方向判定:       {result_mark}')
    else:
        print(f'  方向判定:       ー（ゼロリターンのため判定不可）')

print('-' * 62)
print(f'  推奨ポジション: {pos_direction}')
print(f'  ポジション比率: {position_pct:.1%}（資本の{position_pct*100:.1f}%）')
print(f'  ポジション額:   ${position_value:,.0f}')
print(f'  COMEX契約数:    {n_contracts:.2f}枚（概算）')
print(f'  取引コスト:     ${trade_cost:,.0f}（{COST_BPS}bps）')
print(f'  期待損益（税前）:${expected_pnl_after_cost:+,.0f}')

# 実績ベースのP&L（過去日付のみ）
if actual_return is not None:
    actual_pnl = position_value * actual_return / 100
    actual_pnl_net = actual_pnl - trade_cost
    # ポジション方向を考慮（ロングなら実際のリターン、ショートなら逆）
    if pred_raw > 0:  # ロング
        realized_pnl = actual_pnl - trade_cost
    else:  # ショート
        realized_pnl = -actual_pnl - trade_cost
    print(f'  実現損益（税前）:${realized_pnl:+,.0f}')

print('-' * 62)
print(f'  平均予測誤差:   0.94%（= ${worst_case_loss:,.0f}）')
print(f'  テールリスク:   3.0%（= ${tail_risk_loss:,.0f}）')
print('=' * 62)
print()

# --- フッター ---
if actual_return is None:
    print('※ これは定量モデルの出力であり、投資助言ではありません。')
    print('   過去の実績（的中率60%、Sharpe 2.46）は将来を保証しません。')
    print('   必ずご自身のリスク管理に基づいて判断してください。')
else:
    print('※ 過去日付が指定されたため、実績との照合結果を表示しました。')
    print('   モデルの方向的中率はテストセットで約60%です。')
    print('   個別の予測が外れることは想定内であり、統計的優位性は')
    print('   多数の取引を通じて発現します。')